In [1]:
%load_ext autoreload
%autoreload 2

In [221]:
def train_loop(dataloader, model, loss_fn, optimizer, patience):
    min_valid_loss = np.inf
    max_valid_accuracy = 0
    stop_counter = 0

    for features, labels, adjacency, train_mask, valid_mask in dataloader:
        if stop_counter == patience:
            break

        model.train()
        pred = model(features, adjacency)
        train_pred = pred[train_mask]
        train_labels = labels[train_mask]
        train_loss = loss_fn(train_pred, train_labels)
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            valid_pred = pred[valid_mask]
            valid_labels = labels[valid_mask]
            valid_loss = loss_fn(valid_pred, valid_labels)
            valid_accuracy = valid_pred.argmax(dim=1) == valid_labels
            valid_accuracy = valid_accuracy.type(torch.float).mean().item()

        if valid_loss > min_valid_loss and valid_accuracy < max_valid_accuracy:
            stop_counter += 1
        min_valid_loss = min(min_valid_loss, valid_loss)
        max_valid_accuracy = max(max_valid_accuracy, valid_accuracy)

        print(f"train_loss: {train_loss:>7f}")
        print(f"valid_loss: {valid_loss:>7f}")
        print(f"valid_accuracy: {valid_accuracy:>7f}")


def test_loop(dataloader, model, loss_fn):
    model.eval()
    for features, labels, adjacency, test_mask in dataloader:
        with torch.no_grad():
            pred = model(features, adjacency)
            test_pred = pred[test_mask]
            test_labels = labels[test_mask]
            test_loss = loss_fn(test_pred, test_labels)
            test_accuracy = test_pred.argmax(dim=1) == test_labels
            test_accuracy = test_accuracy.type(torch.float).mean().item()

        print(f"test_loss: {test_loss:>7f}")
        print(f"test_accuracy: {test_accuracy:>7f}")


In [222]:
from dataset import CoraDataset
from torch.utils.data import DataLoader

datadir = "data/cora"
cora_train = CoraDataset(datadir, train=True)
cora_test = CoraDataset(datadir, train=False)
train_dataloader = DataLoader(cora_train)
test_dataloader = DataLoader(cora_test)

In [223]:
import torch
import torch.nn as nn
from model import GraphAttentionNetwork

learning_rate = 0.005
weight_decay = 0.0005
n_epochs = 100_000
patience = 100

model = GraphAttentionNetwork(
    in_features=1432,
    hid_units=8,
    n_classes=7,
    n_heads=(8, 1),
)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model.parameters(), 
    lr=learning_rate, 
    weight_decay=weight_decay,
)

In [225]:
for t in range(n_epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer, patience)
    test_loop(test_dataloader, model, loss_fn)
    print()
print("Done!")

Epoch 1
-------------------------------
train_loss: 2.040562
valid_loss: 1.998334
valid_accuracy: 0.118000
test_loss: 1.911191
test_accuracy: 0.260000

Epoch 2
-------------------------------
train_loss: 1.963241
valid_loss: 1.949209
valid_accuracy: 0.186000
test_loss: 1.845714
test_accuracy: 0.429000

Epoch 3
-------------------------------
train_loss: 1.838173
valid_loss: 1.885617
valid_accuracy: 0.244000
test_loss: 1.779438
test_accuracy: 0.587000

Epoch 4
-------------------------------
train_loss: 1.745909
valid_loss: 1.802392
valid_accuracy: 0.290000
test_loss: 1.718183
test_accuracy: 0.674000

Epoch 5
-------------------------------
train_loss: 1.616834
valid_loss: 1.800024
valid_accuracy: 0.360000
test_loss: 1.658848
test_accuracy: 0.716000

Epoch 6
-------------------------------
train_loss: 1.540220
valid_loss: 1.747678
valid_accuracy: 0.386000
test_loss: 1.599542
test_accuracy: 0.741000

Epoch 7
-------------------------------
train_loss: 1.500523
valid_loss: 1.699731
valid_

KeyboardInterrupt: 

In [23]:
_, _, _, train_mask, _ = list(train_dataloader)[0]
train_mask.shape

torch.Size([1, 2708])

In [44]:
W = torch.ones(3, 4)
x = torch.ones(1, 4)
torch.tensordot(W.unsqueeze(0), x.unsqueeze(0), dims=([2], [1]))

torch.Size([1, 3, 1, 4])

In [34]:
a = torch.ones(5, 2)

In [37]:
a.unsqueeze(dim=0).shape

torch.Size([1, 5, 2])

In [87]:
(cora_train.labels == 6).sum()

tensor(180)

In [92]:
from pyGAT.utils import load_data
adj, features, labels, idx_train, idx_val, idx_test = load_data(path="pyGAT/data/cora/")

Loading cora dataset...


In [101]:
(labels[:140] == 6).sum()

tensor(17)

In [198]:
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
from scipy.sparse.linalg.eigen.arpack import eigsh
import sys

def parse_index_file(filename):
    """Parse index file."""
    index = []
    for line in open(filename):
        index.append(int(line.strip()))
    return index

def sample_mask(idx, l):
    """Create mask."""
    mask = np.zeros(l)
    mask[idx] = 1
    return np.array(mask, dtype=bool)

def load_data(dataset_str): # {'pubmed', 'citeseer', 'cora'}
    """Load data."""
    names = ['x', 'y', 'tx', 'ty', 'allx', 'ally', 'graph']
    objects = []
    for i in range(len(names)):
        with open("GAT/data/ind.{}.{}".format(dataset_str, names[i]), 'rb') as f:
            if sys.version_info > (3, 0):
                objects.append(pkl.load(f, encoding='latin1'))
            else:
                objects.append(pkl.load(f))

    x, y, tx, ty, allx, ally, graph = tuple(objects)
    test_idx_reorder = parse_index_file("GAT/data/ind.{}.test.index".format(dataset_str))
    test_idx_range = np.sort(test_idx_reorder)

    if dataset_str == 'citeseer':
        # Fix citeseer dataset (there are some isolated nodes in the graph)
        # Find isolated nodes, add them as zero-vecs into the right position
        test_idx_range_full = range(min(test_idx_reorder), max(test_idx_reorder)+1)
        tx_extended = sp.lil_matrix((len(test_idx_range_full), x.shape[1]))
        tx_extended[test_idx_range-min(test_idx_range), :] = tx
        tx = tx_extended
        ty_extended = np.zeros((len(test_idx_range_full), y.shape[1]))
        ty_extended[test_idx_range-min(test_idx_range), :] = ty
        ty = ty_extended

    features = sp.vstack((allx, tx)).tolil()
    features[test_idx_reorder, :] = features[test_idx_range, :]
    adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))

    labels = np.vstack((ally, ty))
    labels[test_idx_reorder, :] = labels[test_idx_range, :]

    idx_test = test_idx_range.tolist()
    idx_train = range(len(y))
    idx_val = range(len(y), len(y)+500)

    train_mask = sample_mask(idx_train, labels.shape[0])
    val_mask = sample_mask(idx_val, labels.shape[0])
    test_mask = sample_mask(idx_test, labels.shape[0])

    y_train = np.zeros(labels.shape)
    y_val = np.zeros(labels.shape)
    y_test = np.zeros(labels.shape)
    y_train[train_mask, :] = labels[train_mask, :]
    y_val[val_mask, :] = labels[val_mask, :]
    y_test[test_mask, :] = labels[test_mask, :]

    print(adj.shape)
    print(features.shape)

    return labels, adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask

labels, adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data("cora")


(2708, 2708)
(2708, 1433)


In [117]:
adj.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 1, 0]])

In [133]:
C = np.array(cora_train.adjacency) == np.array(adj.todense() + np.eye(2708), dtype=bool)

In [137]:
C.sum() == 2708 ** 2

True

In [148]:
F1 = np.array(features.todense(), dtype=int)
F2 = np.array(cora_train.features, dtype=int)

In [153]:
type(F1)

numpy.ndarray

In [152]:
F2.sum(1)

array([ 9, 23, 19, ..., 18, 14, 13])

In [158]:
F1.shape

(2708, 1433)

In [192]:
import pandas as pd
words = pd.read_csv("data/cora/words.csv")
np.unique(words.word.values).sum()

1025584

In [196]:
[i for i in range(1432) if i not in words.word.values]

[444]

In [199]:
labels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [206]:
(np.array(pd.get_dummies(cora_train.labels), dtype=int) == np.array(labels, dtype=int)).sum()

18956

In [207]:
2708 * 7

18956

In [216]:
all(x == (140 <= i < 140 + 500) for i, x in enumerate(val_mask))

True